## TextBlob

In [126]:
# https://textblob.readthedocs.io/en/dev/

# How to install TextBlob
#     1. pip install -U textblob
#     2. python -m textblob.download_corpora

from textblob import TextBlob

blob = TextBlob(dfT['text'][0]) # run json-to-pandas

# Part-of-speech Tagging
print blob.tags
print

# Noun Phrase Extraction¶
print blob.noun_phrases
print 

# Tokenization
print blob.words

NameError: name 'dfT' is not defined

In [39]:
# The subjectivity is a float within the range [0.0, 1.0] 
# where 0.0 is very objective and 1.0 is very subjective
for sentence in blob.sentences:
    print sentence.sentiment.polarity
print

# Lemmatize each word
for sentence in blob.sentences:    
    for word in sentence.words:
        print "%s---%s" % (word, word.lemmatize('v')) # 'v' for 'verb'

0.0166666666667

One---One
of---of
China---China
's---'s
first---first
female---female
fighter---fighter
pilots---pilot
was---be
killed---kill
in---in
a---a
training---train
accident---accident
according---accord
to---to
state-run---state-run
media---media
reports…---reports…
https---https
t.co/DoEZLme8Cq---t.co/DoEZLme8Cq


## NLTK

http://www.nltk.org/

In [11]:
import nltk
from nltk.corpus import treebank # to draw a parse tree

sentence = dfT['text'][0] # run json-to-pandas

tokens = nltk.word_tokenize(sentence)

tagged = nltk.pos_tag(tokens)

# Identify named entities - Make parse tree?
# You might need to call nltk.download() and down load some packages
entities = nltk.chunk.ne_chunk(tagged)

In [12]:
# Example from http://streamhacker.com/2010/05/10/text-classification-sentiment-analysis-naive-bayes-classifier/
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
 
def word_feats(words):
    return dict([(word, True) for word in words])
 
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')
 
negfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
posfeats = [(word_feats(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
negcutoff = len(negfeats)*3/4
poscutoff = len(posfeats)*3/4
 
trainfeats = negfeats[:negcutoff] + posfeats[:poscutoff]
testfeats = negfeats[negcutoff:] + posfeats[poscutoff:]
print 'train on %d instances, test on %d instances' % (len(trainfeats), len(testfeats))
 
classifier = NaiveBayesClassifier.train(trainfeats)
print 'accuracy:', nltk.classify.util.accuracy(classifier, testfeats)
classifier.show_most_informative_features()

train on 1500 instances, test on 500 instances
accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


## Supervised Learning w/ manual scripting

In [172]:
# Task 1: Load the texts
import pandas as pd
import glob, os         # for reading all .txt files
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
#from sklearn.base import ClassifierMixin
#from sklearn.dummy import DummyClassifier
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.multiclass import OneVsOneClassifier
#from sklearn.multiclass import OutputCodeClassifier
#from sklearn.neural_network import MLPClassifier
#from sklearn.calibration import CalibratedClassifierCV
#from sklearn.semi_supervised import LabelPropagation
import math
import scipy.sparse as sp

In [24]:
# Read sample texts
sample = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/Turk/Batch_2606965_batch_results.csv') 
sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Neutral':0, 'Negative':-1})

# Split train and test sets
cutoff = int(math.ceil(len(sample.index)*0.92)) # 92%
train = pd.DataFrame({'label':sample['Answer.sentiment'][:cutoff], 'texts':sample['Input.content'][:cutoff]})
test  = pd.DataFrame({'label':sample['Answer.sentiment'][cutoff:], 'texts':sample['Input.content'][cutoff:]})

print train.shape
print test.shape
#sample.head()

(2758, 2)
(239, 2)


In [149]:
# THREE ANSWERS TO ONE
sample = pd.read_csv('/Users/Haru/Downloads/489Project/Turk/6source_results_filtered.csv') 

new_index = range(0, len(sample.index)/3)
new_sample = pd.DataFrame(index=new_index, columns=['texts', 'sources', 'labels'])

text = ""
#acc_value = 0 # accumulated score per post
#num_posts = 0 # number of same text posts
pos = 0 # number of 'positive' per post
neu = 0 # number of 'neutral' per post
neg = 0 # number of 'negative' per post
pd_index = 0

for index, row in sample.iterrows():
    # If this new post is different post, then append what we worked so far
    if text != row['Input.content']:
        if text != "":
            if pos >= 2:
                new_sample.loc[[pd_index], ['labels']] = 1
            elif neu >= 2: # could be commented out for pos/neg
                new_sample.loc[[pd_index], ['labels']] = 0
            elif neg >= 2:
                new_sample.loc[[pd_index], ['labels']] = -1
            else: # 1:1:1
                #continue # discard
                pd_index -= 1 # to keep the same index

            #new_sample.loc[[pd_index], ['score']] = acc_value*1.0/num_posts
            new_sample.loc[[pd_index], ['sources']] = row['Input.source']
            new_sample.loc[[pd_index], ['texts']] = text
            pd_index += 1
            
        # Assign a new post
        text = row['Input.content']
        #acc_value = 0
        #num_posts = 0
        pos = 0
        neu = 0
        neg = 0
        
    if row['Answer.sentiment'] == "Positive":
        #acc_value += 1
        #num_posts += 1
        pos += 1
    elif row['Answer.sentiment'] == "Neutral":
        #num_posts += 1
        neu += 1
    elif row['Answer.sentiment'] == "Negative":
        #acc_value -= 1
        #num_posts += 1
        neg += 1

# not perfect, so we have some extra NaN rows (all should be filled, technically)
new_sample = new_sample.dropna()
        
# change 0.333, 0.666, ... to whole number by x 3
#new_sample.loc[:, 'score'] *= 3
new_sample['labels'] = new_sample['labels'].astype('int')
    
#new_sample['sources'] = new_sample['sources'].map({'msnbc':4, 
#                                                   'cnn':3, 
#                                                   'cbs':2,
#                                                   'usatoday':1,
#                                                   'wsj':-1,
#                                                   'fox':-2})
#new_sample['sources'] = new_sample['sources'].astype('int')

sample = new_sample

print sample
cutoff = int(math.ceil(len(sample.index)*0.9)) # 90%
train = pd.DataFrame({'labels':sample['labels'][:cutoff], 'texts':sample['texts'][:cutoff], 'sources':sample['sources'][:cutoff]})
test  = pd.DataFrame({'labels':sample['labels'][cutoff:], 'texts':sample['texts'][cutoff:], 'sources':sample['sources'][cutoff:]})
print train.shape
print test.shape

                                                  texts   sources  labels
0     It is unusual for a president's children to be...       cbs       1
1     Just hours after Mitt Romney blasted Donald J....       cbs      -1
2     Hillary Clinton wins the state of Washington, ...       fox       1
3     "We have to take the jobs away from other coun...  usatoday      -1
4     Jessica Leeds, 74, of Manhattan, was one of tw...       cbs      -1
5     Ash Carter warns Russia its policy in Syria wi...       cbs      -1
6     "It's Rubio!" Watch Donald J. Trump use a wate...  usatoday       0
7     Online, some Trump supporters discuss monitori...       cnn      -1
8     This mogul once said he's a member of "the luc...       fox       1
9     Live: Hillary Clinton speaks at the NAACP Conv...       cbs       0
10    BREAKING: Hillary Clinton has chosen Virginia ...  usatoday       0
11    The billboard translated reads: "Donald Trump,...       cnn      -1
12    How weird was the world of polit

In [194]:
# classifying candidates
def classify_post (row):
   status = row['texts'].lower()
   if (('donald' in status) or ('trump' in status)) and (('hillary' in status) or ('clinton' in status)) :
      return 'other'
   elif ('donald' in status) or ('trump' in status) :
      return 'trump'
   elif ('hillary' in status) or ('clinton' in status) :
      return 'clinton'
   return 'other'

sample['candidates'] = sample.apply (lambda row: classify_post (row),axis=1)

cutoff = int(math.ceil(len(sample.index)*0.9)) # 90%
train = pd.DataFrame({'labels':sample['labels'][:cutoff], 
                      'texts':sample['texts'][:cutoff], 
                      'sources':sample['sources'][:cutoff], 
                      'candidates':sample['candidates'][:cutoff]})
test  = pd.DataFrame({'labels':sample['labels'][cutoff:], 
                      'texts':sample['texts'][cutoff:], 
                      'sources':sample['sources'][cutoff:], 
                      'candidates':sample['candidates'][cutoff:]})

print sample.head()

                                               texts   sources  labels  \
0  It is unusual for a president's children to be...       cbs       1   
1  Just hours after Mitt Romney blasted Donald J....       cbs      -1   
2  Hillary Clinton wins the state of Washington, ...       fox       1   
3  "We have to take the jobs away from other coun...  usatoday      -1   
4  Jessica Leeds, 74, of Manhattan, was one of tw...       cbs      -1   

  candidates  
0      trump  
1      trump  
2    clinton  
3      trump  
4      trump  


In [ ]:
#for j in range(0,2): # stop_words: k=0 'engl' k=1 none

for h in range(5,31,2): # min_df from 0.05 to 0.30, adding 0.02 every time
    for i in range(95,69,-2): # max_df from 0.95 to 0.70, -0.02 every time
        for k in range(0,2): # k=0 CountVectorizer (count), k=1 TfidfVectorizer (weighed)
            for l in range(0,25): # different classificers

                if k==0:
                    tf_vectorizer = CountVectorizer(max_df=i/100.0, min_df=h/100.0)
                elif k==1:
                    tf_vectorizer = TfidfVectorizer(max_df=i/100.0, min_df=h/100.0)

                train_text_tf_  = tf_vectorizer.fit_transform(train['texts'].values)
                test_text_tf_  = tf_vectorizer.transform(test['texts'].values)

                train_source_tf_ = tf_vectorizer.fit_transform(train['sources'].values)
                test_source_tf_ = tf_vectorizer.transform(test['sources'].values)

                train_candidate = tf_vectorizer.fit_transform(train['candidates'].values)
                test_candidate = tf_vectorizer.transform(test['candidates'].values)
                
                train_combined = sp.hstack([train_text_tf_, train_source_tf_], format='csr')
                test_combined = sp.hstack([test_text_tf_, test_source_tf_], format='csr')

                train_combined = sp.hstack([train_combined, train_candidate], format='csr')
                test_combined = sp.hstack([test_combined, test_candidate], format='csr')
                
                # CHANGE ARGS TOO
                if l==0:
                    clf = MultinomialNB()
                elif l==1:
                    clf = GaussianNB()
                elif l==2:
                    clf = BernoulliNB()
                elif l==3:
                    clf = LogisticRegression()
                elif l==4:
                    clf = LogisticRegressionCV()
                elif l==5:
                    clf = SGDClassifier()
                elif l==6:
                    clf = Ridge()
                elif l==7:
                    clf = RidgeClassifier()
                elif l==8:
                    clf = RidgeClassifierCV()
                elif l==9:
                    clf = ElasticNet()
                elif l==10:
                    clf = LassoLars()
                elif l==11:
                    clf = PassiveAggressiveClassifier()
                elif l==12:
                    clf = SVC()
                elif l==13:
                    clf = LinearSVC()
                elif l==14:
                    clf = KNeighborsClassifier()
                elif l==15:
                    clf = NearestCentroid()
                elif l==16:
                    clf = GradientBoostingClassifier()
                elif l==17:
                    clf = BaggingClassifier()
                elif l==18:
                    clf = RandomForestClassifier()
                elif l==19:
                    clf = AdaBoostClassifier()
                elif l==20:
                    clf = ExtraTreesClassifier()
                elif l==21:
                    clf = GradientBoostingClassifier()
                elif l==22:
                    clf = VotingClassifier(estimators=
                            [('lr', LogisticRegression(random_state=1)), 
                             ('rf', RandomForestClassifier(random_state=1)), 
                             ('gnb', GaussianNB())])
                elif l==23:
                    clf = ExtraTreeClassifier()
                elif l==24:
                    clf = DecisionTreeClassifier()

                if l==1 or l==10 or l==22:
                    clf.fit(train_combined.toarray(), train['labels'])
                    score = clf.score(test_combined.toarray(), test['labels'])
                    if score > 0.60:
                        print "%d %d %d %d\t %.4f" % (h,i,k,l,score)
                    else:
                        print "%d %d %d %d" % (h,i,k,l)
                else:
                    clf.fit(train_combined, train['labels'])
                    score = clf.score(test_combined.toarray(), test['labels'])
                    if score > 0.60:
                        print "%d %d %d %d\t %.4f" % (h,i,k,l,score)
                    else:
                        print "%d %d %d %d" % (h,i,k,l)

5 95 5 0 0
5 95 5 0 1
5 95 5 0 2
5 95 5 0 3
5 95 5 0 4
5 95 5 0 5
5 95 5 0 6
5 95 5 0 7
5 95 5 0 8
5 95 5 0 9
5 95 5 0 10
5 95 5 0 11
5 95 5 0 12
5 95 5 0 13
5 95 5 0 14
5 95 5 0 15
5 95 5 0 16
5 95 5 0 17
5 95 5 0 18
5 95 5 0 19
5 95 5 0 20
5 95 5 0 21
5 95 5 0 22
5 95 5 0 23
5 95 5 0 24
5 95 5 1 0
5 95 5 1 1
5 95 5 1 2
5 95 5 1 3
5 95 5 1 4
5 95 5 1 5
5 95 5 1 6
5 95 5 1 7
5 95 5 1 8
5 95 5 1 9
5 95 5 1 10
5 95 5 1 11
5 95 5 1 12
5 95 5 1 13
5 95 5 1 14
5 95 5 1 15
5 95 5 1 16
5 95 5 1 17
5 95 5 1 18
5 95 5 1 19
5 95 5 1 20
5 95 5 1 21
5 95 5 1 22
5 95 5 1 23
5 95 5 1 24
5 93 5 0 0
5 93 5 0 1
5 93 5 0 2
5 93 5 0 3
5 93 5 0 4
5 93 5 0 5
5 93 5 0 6
5 93 5 0 7
5 93 5 0 8
5 93 5 0 9
5 93 5 0 10
5 93 5 0 11
5 93 5 0 12
5 93 5 0 13
5 93 5 0 14
5 93 5 0 15
5 93 5 0 16
5 93 5 0 17
5 93 5 0 18
5 93 5 0 19
5 93 5 0 20
5 93 5 0 21
5 93 5 0 22
5 93 5 0 23
5 93 5 0 24
5 93 5 1 0
5 93 5 1 1
5 93 5 1 2
5 93 5 1 3
5 93 5 1 4
5 93 5 1 5
5 93 5 1 6
5 93 5 1 7
5 93 5 1 8
5 93 5 1 9
5 93 5 1 10
5 93 5 1

Out of the 20,000+ possibilities and running 8000+ cases, <br>
It never hit 0.7.. ->> For train:test = 1:1 size

Train:test = 10:1 size. I do get up to 0.8+ <br>
Now we also have up to (more reasonable) 14400 cases (all expected to run)

## Supervised Learning w/ TPOT

In [182]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import csv
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [195]:
#for i in range(0,2): # 'Negative' from sample = -1 or 0
#  if i==0:
#      sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Negative':-1})
#  elif i==1:
#      sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Negative':0})

for j in range(5,21,2): # min_df
    for k in range(80,101,2): # max_df
        for l in range(0,2): # l=0 CountVectorizer (count), l=1 TfidfVectorizer (weighed)
            for m in range(50,101,10): # train:test = m/100 : (1-m/100) so 50:50 to 90:10
                for n in range(5,7): # generation (# of TPOT iteration)
                    for p in range(20,26,5): # pop_size p 
                        for q in range(10,12): # k-fold number
                            print ""    

                            #sample = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/sample_data_results.csv') 

                            # label ('positive','Negative') ->> (#,#) (e.g. (1,-1) or (1,0))
                            #sample['Answer.sentiment'] = sample['Answer.sentiment'].map({'Positive':1, 'Neutral':0, 'Negative':-1})

                            if l==0:
                                tf_vectorizer = CountVectorizer(min_df=j/100.0, max_df=k/100.0)
                            elif l==1:
                                tf_vectorizer = TfidfVectorizer(min_df=j/100.0, max_df=k/100.0)

                            #sample_input_tf  = tf_vectorizer.fit_transform(sample['Input.content'].values)
                            text_tf_  = tf_vectorizer.fit_transform(sample['texts'].values)
                            source_tf_ = tf_vectorizer.fit_transform(sample['sources'].values)
                            candidates_tf_ = tf_vectorizer.fit_transform(sample['candidates'].values)
                            combined = sp.hstack([text_tf_, source_tf_], format='csr')
                            combined = sp.hstack([combined, candidates_tf_], format='csr')

                            X_train, X_test, y_train, y_test = train_test_split(combined, sample['labels'].values,
                                                                train_size=m/100.0, test_size=(1-m/100.0))#, random_state=)

                            # Official website example: gen=5, pop_size=20, verbo=2
                            tpot = TPOTClassifier(generations=n, population_size=p, num_cv_folds=q,
                                                  verbosity=2)
                            tpot.fit(X_train, y_train)
                            print(tpot.score(X_test, y_test))
                            print "%d %d %d %d %d %d %d %d" % (i,j,k,l,m,n,p,q)

Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

Optimization Progress:  17%|█▋        | 20/120 [01:41<03:51,  2.32s/pipeline]

Generation 1 - Current best internal CV score: 0.575650487096


Optimization Progress:  32%|███▏      | 38/120 [01:54<01:01,  1.33pipeline/s]

Generation 2 - Current best internal CV score: 0.575650487096


...

Generation 3 - Current best internal CV score: 0.550133066605


Optimization Progress:  68%|██████▊   | 81/120 [20:00<12:27, 19.16s/pipeline]

Generation 4 - Current best internal CV score: 0.550133066605



GP closed prematurely - will use current best pipeline


KeyboardInterrupt: 

In [154]:
# Just to run it twice

for i in range(0,2): # min_df
    if i==0:
        tf_vectorizer = CountVectorizer(max_df=0.8, min_df=0.1)
    elif i==1:
        tf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.1)

    text_tf_  = tf_vectorizer.fit_transform(sample['texts'].values)
    source_tf_ = tf_vectorizer.fit_transform(sample['sources'].values)

    combined = sp.hstack([text_tf_, source_tf_], format='csr')

    X_train, X_test, y_train, y_test = train_test_split(combined, sample['labels'].values,
                                        train_size=0.92, test_size=0.08)#, random_state=)

    # Official website example: gen=5, pop_size=20, verbo=2
    tpot = TPOTClassifier(generations=5, population_size=20, verbosity=2)
    tpot.fit(X_train, y_train)
    print(tpot.score(X_test, y_test))
    #print "%d %d %d %d %d %d %d %d" % (i,j,k,l,m,n,p,q)

Optimization Progress:  17%|█▋        | 20/120 [00:48<06:32,  3.92s/pipeline]

Generation 1 - Current best internal CV score: 0.519661680143



GP closed prematurely - will use current best pipeline

Best pipeline: RandomForestClassifier(input_matrix)


Optimization Progress:   0%|          | 0/120 [00:00<?, ?pipeline/s]

0.454433137623


...

Optimization Progress:  31%|███       | 37/120 [02:45<07:48,  5.64s/pipeline]

Generation 2 - Current best internal CV score: 0.520328662831


Optimization Progress:  48%|████▊     | 57/120 [07:13<19:56, 18.98s/pipeline]

Generation 3 - Current best internal CV score: 0.523811364363



GP closed prematurely - will use current best pipeline

Best pipeline: LinearSVC(input_matrix, 0.57999999999999996, 2, False)
0.544801358419


In [141]:
# Lemmanize the sentences (e.g. is, are, am ->> be)

new_index = range(0, len(sample.index))
new_sample = pd.DataFrame(index=new_index, columns=("texts", "sources", "labels"))
i=0
for text in sample['texts']:
    blob = TextBlob(text)
    newtexts = ""

    for sentence in blob.sentences:
        newtext = ""
#        print sentence.dict

        for word in sentence.words:
            newtext += " " + word.lemmatize('v') # 'v' for 'verb'

        newtexts += newtext
    new_sample.loc[i] = newtexts
    i += 1

i=0
for answer in sample['sources']: # updating answers
    new_sample['sources'].loc[i] = answer
    i += 1
i=0
for answer in sample['labels']: # updating answers
    new_sample['labels'].loc[i] = answer
    i += 1
    
save_sample = sample
sample = new_sample
print sample

                                                  texts   sources labels
0      It be unusual for a president 's children to ...       cbs      1
1      Just hours after Mitt Romney blast Donald J T...       cbs     -1
2      Hillary Clinton win the state of Washington C...       fox      1
3      We have to take the job away from other count...  usatoday     -1
4      Jessica Leeds 74 of Manhattan be one of two w...       cbs     -1
5      Ash Carter warn Russia its policy in Syria wi...       cbs     -1
6      It 's Rubio Watch Donald J Trump use a water ...  usatoday      0
7      Online some Trump supporters discuss monitor ...       cnn     -1
8      This mogul once say he 's a member of the luc...       fox      1
9      Live Hillary Clinton speak at the NAACP Conve...       cbs      0
10     BREAKING Hillary Clinton have choose Virginia...  usatoday      0
11     The billboard translate read Donald Trump he ...       cnn     -1
12     How weird be the world of politics in 2015 W

CountVectorizer, Pos=1 Neg=-1, train:test=75:25, size=872 in TPOT = 0.78 <br>
CountVectorizer, Pos=1 Neg=-1, train:test=92:08, size=872 in TPOT = 0.78 <br>
CountVectorizer, Pos=1 Neg =0, train:test=92:08, size=872 in TPOT = 0.86 <br>



## Unsupervised Learning

In [42]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()


from sklearn.decomposition import NMF, LatentDirichletAllocation

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   #max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(train['texts'].values)

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                #max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(train['texts'].values)

lda = LatentDirichletAllocation(#n_topics=n_topics, 
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

lda.fit(tf)

tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

Topic #0:
trump donald presidency nigel farage loathsome creature calls brexit leader obama inside national guru security mind change climate experts increasingly
Topic #1:
trump new york protesters win yorkers database tweets unfair hours info light calling mayor nyc deleted undocumented praise calls denounce
Topic #2:
rt help trump author historian lady future shyness cnnnewsroom detect melania speaker ryan deportation erecting force planning paul cnnpolitics allies
Topic #3:
america like americans hope trump promised watching does reid white fear feel tears innocent celebrate nationalists breath says deep everybody
Topic #4:
trump obamacare pulling going pol rug backing repeal day interview pledge appeared country end needs replace open agreed work democrats
Topic #5:
trump says kelly denies advance saw megyn debate report book question children gets advising rudy cnnsotu government giuliani jobs lead
Topic #6:
president cast ballots million unqualified candidates voters nearly 18 l

## Ensemble & Bagging (Bootstrap AGgregating)

In [104]:
# Nope... http://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/

## So text extraction + ..?

In [108]:
# http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

In [48]:
# Task 1: Load the texts
import pandas as pd
import glob, os         # for reading all .txt files
import csv
import numpy as np
from textblob import TextBlob # use kernel Python[Root]

cutoff = 436

# Read sample texts
sample = pd.read_csv('/Users/Haru/Documents/! College/4. Fall 2016/489Project/sample_data_results.csv') 

# Lemmatize
new_sample = pd.DataFrame(columns=("Input.content", "Answer.sentiment"))
i=0
for text in sample['Input.content']:
    blob = TextBlob(text)
    newtexts = ""

    for sentence in blob.sentences:
        newtext = ""
#        print sentence.dict

        for word in sentence.words:
            newtext += " " + word.lemmatize('v') # 'v' for 'verb'

        newtexts += newtext
        new_sample.loc[i] = newtexts
        i += 1

i=0
for answer in sample['Answer.sentiment']: # updating answers
    new_sample['Answer.sentiment'].loc[i] = answer
    i += 1
    
sample = new_sample

In [49]:
train = pd.DataFrame({'label':sample['Answer.sentiment'][:cutoff], 'texts':sample['Input.content'][:cutoff]})
test  = pd.DataFrame({'label':sample['Answer.sentiment'][cutoff:], 'texts':sample['Input.content'][cutoff:]})

In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Task 4: feature engineering
for h in range(1,30): # min_df
    for i in range(h+5,100): # max_df
        for j in range(0,2): # stop_words: k=0 'engl' k=1 none
            for k in range(0,2): # k=0 CountVectorizer (count), k=1 TfidfVectorizer (weighed)
                for l in range(0,3): # l=0 MultinomialNB, l=1 GaussianNB, l=2 BernoulliNB              

                    if j==0 & k==0:
                        tf_vectorizer = CountVectorizer(max_df=i/100.0, min_df=h/100.0, stop_words='english')
                    elif j==1 & k==0:
                        tf_vectorizer = CountVectorizer(max_df=i/100.0, min_df=h/100.0)
                    elif j==0 & k==1:
                        tf_vectorizer = TfidfVectorizer(max_df=i/100.0, min_df=h/100.0, stop_words='english')
                    elif j==1 & k==1:
                        tf_vectorizer = TfidfVectorizer(max_df=i/100.0, min_df=h/100.0)

                    train_tf_ = tf_vectorizer.fit_transform(train['texts'].values)
                    test_tf_  = tf_vectorizer.transform(test['texts'].values)

                    if l==0:
                        clf = MultinomialNB()
                    elif l==1:
                        clf = GaussianNB()
                    elif l==2:
                        clf = BernoulliNB()

                    if l==0 | l==2:
                        clf.fit(train_tf_, train['label'])
                        print "%d %d %d %d %d - %.4f" % (h,i,j,k,l,clf.score(test_tf_, test['label']))
                    elif l==1:
                        clf.fit(train_tf_.toarray(), train['label'])
                        print "%d %d %d %d %d - %.4f" % (h,i,j,k,l,clf.score(test_tf_.toarray(), test['label']))

1 6 0 0 1 - 0.7778
1 6 0 0 2 - 0.6389
1 6 0 1 1 - 0.7778
1 6 0 1 2 - 0.6389
1 6 1 0 1 - 0.7778
1 6 1 0 2 - 0.6389
1 6 1 1 1 - 0.7222
1 6 1 1 2 - 0.6944
1 7 0 0 1 - 0.8056
1 7 0 0 2 - 0.6389
1 7 0 1 1 - 0.8056
1 7 0 1 2 - 0.6389
1 7 1 0 1 - 0.8056
1 7 1 0 2 - 0.6389
1 7 1 1 1 - 0.7222
1 7 1 1 2 - 0.6389
1 8 0 0 1 - 0.8056
1 8 0 0 2 - 0.6389
1 8 0 1 1 - 0.8056
1 8 0 1 2 - 0.6389
1 8 1 0 1 - 0.8056
1 8 1 0 2 - 0.6389
1 8 1 1 1 - 0.7222
1 8 1 1 2 - 0.6389
1 9 0 0 1 - 0.8056
1 9 0 0 2 - 0.5833
1 9 0 1 1 - 0.8056
1 9 0 1 2 - 0.5833
1 9 1 0 1 - 0.8056
1 9 1 0 2 - 0.5833
1 9 1 1 1 - 0.7222
1 9 1 1 2 - 0.6389
1 10 0 0 1 - 0.8056
1 10 0 0 2 - 0.6389
1 10 0 1 1 - 0.8056
1 10 0 1 2 - 0.6389
1 10 1 0 1 - 0.8056
1 10 1 0 2 - 0.6389
1 10 1 1 1 - 0.7222
1 10 1 1 2 - 0.6944
1 11 0 0 1 - 0.8056
1 11 0 0 2 - 0.6389
1 11 0 1 1 - 0.8056
1 11 0 1 2 - 0.6389
1 11 1 0 1 - 0.8056
1 11 1 0 2 - 0.6389
1 11 1 1 1 - 0.7222
1 11 1 1 2 - 0.6944
1 12 0 0 1 - 0.8056
1 12 0 0 2 - 0.6389
1 12 0 1 1 - 0.8056
1 12 0 1 2 -

KeyboardInterrupt: 